In [47]:
from skylark import skylark_root
from skylark.chunk import ChunkState
import pandas as pd
import json
log_df = pd.read_pickle('/tmp/chunk_status_log.pkl')
valid_states = [ChunkState.download_in_progress, ChunkState.downloaded, ChunkState.upload_in_progress, ChunkState.upload_complete]
log_df = log_df[log_df['state'].isin(valid_states)]
log_df

,chunk_id,state,time,region,instance,receiver_id,sender_id
3,0,ChunkState.download_in_progress,2022-02-17 02:20:53.070778,aws:us-east-1,0,random,NaN
10,1,ChunkState.download_in_progress,2022-02-17 02:20:53.073511,aws:us-east-1,0,random,NaN
12,2,ChunkState.download_in_progress,2022-02-17 02:20:53.074207,aws:us-east-1,0,random,NaN
15,3,ChunkState.download_in_progress,2022-02-17 02:20:53.074928,aws:us-east-1,0,random,NaN
17,4,ChunkState.download_in_progress,2022-02-17 02:20:53.076053,aws:us-east-1,0,random,NaN
...,...,...,...,...,...,...,...
12282,933,ChunkState.upload_in_progress,2022-02-17 02:21:09.844770,aws:us-west-2,0,NaN,save_local
12283,933,ChunkState.upload_complete,2022-02-17 02:21:09.845029,aws:us-west-2,0,NaN,save_local
12284,901,ChunkState.downloaded,2022-02-17 02:21:10.052315,aws:us-west-2,0,receiver:15,NaN
12286,901,ChunkState.upload_in_progress,2022-02-17 02:21:10.243196,aws:us-west-2,0,NaN,save_local


In [49]:
# list of TraceEvent objects
events = []
start_time = log_df.time.min()
for _, row in log_df.iterrows():
    if row['receiver_id'] == 'random':
        continue
    if row['state'] == ChunkState.download_in_progress:
        events.append({
            'name': f"Download {row['chunk_id']}",
            'ts': (row['time'] - start_time).total_seconds() * 1e6,
            'pid': f"{row['region']}:{row['instance']}",
            "tid": row['receiver_id'],
            'ph': 'B'
        })
    elif row['state'] == ChunkState.downloaded:
        events.append({
            'name': f"Download {row['chunk_id']}",
            'ts': (row['time'] - start_time).total_seconds() * 1e6,
            'pid': f"{row['region']}:{row['instance']}",
            "tid": row['receiver_id'],
            'ph': 'E'
        })
    elif row['state'] == ChunkState.upload_in_progress:
        events.append({
            'name': f"Upload {row['chunk_id']}",
            'ts': (row['time'] - start_time).total_seconds() * 1e6,
            'pid': f"{row['region']}:{row['instance']}",
            "tid": row['sender_id'],
            'ph': 'B'
        })
    elif row['state'] == ChunkState.upload_complete:
        events.append({
            'name': f"Upload {row['chunk_id']}",
            'ts': (row['time'] - start_time).total_seconds() * 1e6,
            'pid': f"{row['region']}:{row['instance']}",
            "tid": row['sender_id'],
            'ph': 'E'
        })

with (skylark_root / "data" / "profile.json").open('w') as f:
    f.write(json.dumps(events))